In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

squad = load_dataset("squad")
#squad = load_dataset("squad_v2")

Reusing dataset squad (/home/chang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import BigBirdTokenizerFast
import torch

tokenizer = BigBirdTokenizerFast.from_pretrained("abhinavkulkarni/bigbird-roberta-base-finetuned-squad")

In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
    

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
print(tokenized_squad['train'][0])

{'input_ids': [65, 1776, 4251, 851, 363, 5384, 5436, 8011, 1757, 388, 1349, 3466, 388, 507, 555, 9007, 4982, 131, 66, 17441, 20322, 112, 363, 1625, 569, 358, 7936, 2196, 114, 418, 4953, 363, 8875, 11920, 439, 3970, 29601, 419, 358, 10962, 15308, 387, 363, 5384, 5436, 114, 34629, 388, 2267, 387, 363, 8875, 11920, 391, 6577, 441, 112, 419, 358, 15418, 15308, 387, 2052, 452, 5202, 611, 49410, 452, 363, 8278, 467, 37623, 679, 1316, 2286, 16644, 2617, 2012, 7507, 385, 363, 8875, 11920, 419, 363, 32621, 4071, 387, 363, 17481, 8995, 114, 34629, 2258, 363, 37893, 4071, 419, 363, 2003, 17732, 112, 358, 38020, 1396, 387, 11544, 391, 14681, 114, 733, 419, 358, 30170, 387, 363, 1137, 17732, 480, 507, 555, 9007, 112, 4982, 911, 363, 5384, 5436, 403, 2097, 49389, 4221, 385, 9382, 6307, 425, 5958, 1507, 29704, 7697, 388, 1349, 3466, 114, 1730, 363, 987, 387, 363, 1489, 3809, 458, 493, 388, 358, 1378, 1728, 427, 20518, 933, 614, 25928, 391, 363, 3662, 31491, 929, 419, 358, 2930, 112, 3761, 7916, 15308

In [9]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate 

  0%|          | 0/11 [00:00<?, ?ba/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate 

In [6]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

2022-05-27 19:25:37.492232: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [7]:
from transformers import BigBirdForQuestionAnswering, TrainingArguments, Trainer

model = BigBirdForQuestionAnswering.from_pretrained("abhinavkulkarni/bigbird-roberta-base-finetuned-squad").cuda()

In [8]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_bigbird",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/home/chang/anaconda3/envs/domain/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 87599
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 32850
Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
1,0.946400,0.943572
2,0.691500,0.886037
3,0.518700,0.967431


Saving model checkpoint to ./fine_tuned_bigbird/checkpoint-500
Configuration saved in ./fine_tuned_bigbird/checkpoint-500/config.json
Model weights saved in ./fine_tuned_bigbird/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./fine_tuned_bigbird/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./fine_tuned_bigbird/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./fine_tuned_bigbird/checkpoint-1000
Configuration saved in ./fine_tuned_bigbird/checkpoint-1000/config.json
Model weights saved in ./fine_tuned_bigbird/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./fine_tuned_bigbird/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./fine_tuned_bigbird/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./fine_tuned_bigbird/checkpoint-1500
Configuration saved in ./fine_tuned_bigbird/checkpoint-1500/config.json
Model weights saved in ./fine_tuned_bigbird/checkpoint-1500/pytorch_model.bin
tok

TrainOutput(global_step=32850, training_loss=0.7782535319567815, metrics={'train_runtime': 6936.3076, 'train_samples_per_second': 37.887, 'train_steps_per_second': 4.736, 'total_flos': 7.24817216479703e+16, 'train_loss': 0.7782535319567815, 'epoch': 3.0})

In [9]:
torch.save(model.state_dict(), './fine_bigbird.pt')

In [1]:
from transformers import BigBirdTokenizerFast, BigBirdForQuestionAnswering
import torch
tokenizer = BigBirdTokenizerFast.from_pretrained("abhinavkulkarni/bigbird-roberta-base-finetuned-squad")
model = BigBirdForQuestionAnswering.from_pretrained("abhinavkulkarni/bigbird-roberta-base-finetuned-squad").cuda()
model.load_state_dict(torch.load('./fine_bigbird.pt'))

<All keys matched successfully>

In [2]:
import pandas
import time

bible_data = pandas.read_csv('./bible_summary.csv', header=None)
bible_text = bible_data[3].tolist()[1:]

context = ""
for i in range(len(bible_text)):
    context += bible_text[i]

In [3]:
question = "Who did the God blessed?"
n = len(context)
start = 0
maxtorch = 0
answer_start = 0
answer_end = 0
answer = ""
flag = True
index = 0
start_time = time.time()
while flag:
    index += 1
    print(index)
    end = start + 15000
    if end >= n:
        flag = False
        end = n-1
    while True:
        if context[end] == "." or context[end] == "?" or context[end] == "!":
            break
        end -= 1
    inputs = tokenizer.encode_plus(question, context[start:end+1], truncation=True, return_tensors='pt')
    inputs.to('cuda')
    
    outputs = model(**inputs)
    even = (int(torch.max(outputs[0])) + int(torch.max(outputs[1]))) / 2
    if even > maxtorch:
        maxtorch = even
        answer_start = torch.argmax(outputs[0])  
        answer_end = torch.argmax(outputs[1]) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    
    #answer_start = torch.argmax(outputs[0])  
    #answer_end = torch.argmax(outputs[1]) + 1
    
    #answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    start = end + 1
    if context[start] == " ":
        start += 1
end_time = time.time()
exec_time = end_time - start_time
print("exec_time:{:.5f}sec".format(exec_time))
print(answer)
torch.cuda.empty_cache()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
exec_time:47.13561sec
Obed-Edom
